This was inspired by eduonix and kavgan. it  is a text message spam\ham classifier. it achieves a 98.4 % accuracy, the preprocessing and processing stages will be refined in the near future. 

In [1]:
#calling dependencies

import sys
import nltk
import pandas
import sklearn
import numpy

print('Python:{}',format(sys.version))
print('NLTK:{}',format(nltk.__version__))
print('Pandas:{}',format(pandas.__version__))
print('Scikit-learn:{}',format(sklearn.__version__))
print('Numpy:{}',format(numpy.__version__))

      


Python:{} 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
NLTK:{} 3.4.5
Pandas:{} 0.25.1
Scikit-learn:{} 0.21.3
Numpy:{} 1.16.5


loading the Dataset(dataset downloaded from Kaggle https://archive.ics.uci.edu/ml/datasets/sms+spam+collection

In [2]:
import pandas as pd
import numpy as np

#load  dataset in workable format

df = pd.read_table('SMSSpamCollection',header = None, encoding = 'utf-8')




In [3]:
#basic info about the dataset

print(df.info())
print(df.head())
print(
)
classes = df[0]
print(classes.value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
0    5572 non-null object
1    5572 non-null object
dtypes: object(2)
memory usage: 87.2+ KB
None
      0                                                  1
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...

ham     4825
spam     747
Name: 0, dtype: int64


2. preprocess the data

convert class labels to binary values 0=ham, 1= spam

In [4]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
Y = encoder.fit_transform(classes)
print(classes[:10])
print(Y[:10])

0     ham
1     ham
2    spam
3     ham
4     ham
5    spam
6     ham
7     ham
8    spam
9    spam
Name: 0, dtype: object
[0 0 1 0 0 1 0 0 1 1]


store sms message data

In [5]:
sms_messages = df[1]
print(sms_messages[:10])

0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
2    Free entry in 2 a wkly comp to win FA Cup fina...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
5    FreeMsg Hey there darling it's been 3 week's n...
6    Even my brother is not like to speak with me. ...
7    As per your request 'Melle Melle (Oru Minnamin...
8    WINNER!! As a valued network customer you have...
9    Had your mobile 11 months or more? U R entitle...
Name: 1, dtype: object


use regexp to replace email addresses, url's, phone numbers, other numbers, symbols

replace email addresses with 'emailadd'


In [6]:
#ue regexp to replace urls, email addresses and currencies
processed = sms_messages.str.replace(r'^,+@[^\,],*\,[a-z]{2,}$', 'emilad')  

processed = processed.str.replace(r'^http\://[a-zA-Z0-9\-\,]+\,[a-zA-Z]{2,3}(/\S*)?$', 'wbadd')

processed = processed.str.replace(r'£|\$', 'cshsgn')

processed = processed.str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$','pnumbr')
                                  
processed = processed.str.replace(r'\d+(\.\d+)?', 'nber')


In [7]:
#remove punctuation

processed = processed.str.replace(r'[^\w\d\s]','')

#change to lower case
processed = processed.str.lower()
print(processed)

0       go until jurong point crazy available only in ...
1                                 ok lar joking wif u oni
2       free entry in nber a wkly comp to win fa cup f...
3             u dun say so early hor u c already then say
4       nah i dont think he goes to usf he lives aroun...
                              ...                        
5567    this is the nbernd time we have tried nber con...
5568                  will ü b going to esplanade fr home
5569    pity  was in mood for that soany other suggest...
5570    the guy did some bitching but i acted like id ...
5571                            rofl its true to its name
Name: 1, Length: 5572, dtype: object


In [8]:
#tokenize and remove stop words

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.probability import FreqDist

#creating bag of words

all_words = []


for message in processed:
    sentences = sent_tokenize(message)
    for words in sentences:
        w= word_tokenize(words)
        for c in w:
            all_words.append(c)
    
all_words = FreqDist(all_words)

  



In [9]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

temp_processed=[token for token in all_words if token not in stop_words]
temp_processed = FreqDist(temp_processed)
print('number of words: {}',format(len(temp_processed)))
print('Most common words: {}',format(temp_processed.most_common(50)))

number of words: {} 8740
Most common words: {} [('go', 1), ('jurong', 1), ('point', 1), ('crazy', 1), ('available', 1), ('bugis', 1), ('n', 1), ('great', 1), ('world', 1), ('la', 1), ('e', 1), ('buffet', 1), ('cine', 1), ('got', 1), ('amore', 1), ('wat', 1), ('ok', 1), ('lar', 1), ('joking', 1), ('wif', 1), ('u', 1), ('oni', 1), ('free', 1), ('entry', 1), ('nber', 1), ('wkly', 1), ('comp', 1), ('win', 1), ('fa', 1), ('cup', 1), ('final', 1), ('tkts', 1), ('nberst', 1), ('may', 1), ('text', 1), ('receive', 1), ('questionstd', 1), ('txt', 1), ('ratetcs', 1), ('apply', 1), ('nberovernbers', 1), ('dun', 1), ('say', 1), ('early', 1), ('hor', 1), ('c', 1), ('already', 1), ('nah', 1), ('dont', 1), ('think', 1)]


In [10]:
#use the most common features to train
word_features = list(all_words.keys())[:1500]
#define a find features function
def find_features(message):
    words=word_tokenize(message)
    features = {}
    for word in word_features:
        features[word] = (word in words)
        
    return features
#view results

features = find_features(processed[0])
for key,value in features.items():
    if value == True:
        print (key)

go
until
jurong
point
crazy
available
only
in
bugis
n
great
world
la
e
buffet
cine
there
got
amore
wat


In [11]:
#find features for all messages
messages = list(zip(processed, Y))


#define seed for reproducibility
seed=1
np.random.seed = seed
np.random.shuffle(messages)

featuresets = [(find_features(text),label) for (text, label) in messages]

In [12]:
#splitting the featuresets into training and testing
from sklearn import model_selection

#split the data into training and testing
training, testing = model_selection.train_test_split(featuresets, test_size =0.20, random_state=seed)


In [13]:
print(len(training))
print(len(testing))

4457
1115


Scikit learn Classifiers with NLTK

In [14]:
#we will import some algorithms from sklearn to use as well as some performance metrics
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.svm import SVC

model= SklearnClassifier(SVC(kernel = 'linear'))


#train the model
model.train(training)

#test on testing dataset
accuracy = nltk.classify.accuracy(model, testing)*100

print("SVC accuracy: {}". format(accuracy))

SVC accuracy: 98.11659192825111


In [15]:

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

#define models to train
names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", 
         "SGD Classifier", "Naive Bayes", "SVM Linear"]

import warnings
warnings.filterwarnings("ignore")

classifiers= [
    DecisionTreeClassifier(),
    KNeighborsClassifier(),
    LogisticRegression(),
    RandomForestClassifier(),
    MultinomialNB(),
    SGDClassifier(max_iter = 100),
    SVC(kernel = 'linear')
]

models = list(zip(names, classifiers))

for name, model in models:
    nltk_model =SklearnClassifier(model)
    nltk_model.train(training)
    accuracy = nltk.classify.accuracy(nltk_model, testing)*100
    print("{} Accuracy: {}".format(name, accuracy))

K Nearest Neighbors Accuracy: 95.96412556053812
Decision Tree Accuracy: 94.43946188340807
Random Forest Accuracy: 98.65470852017937
Logistic Regression Accuracy: 97.57847533632287
SGD Classifier Accuracy: 98.02690582959642
Naive Bayes Accuracy: 98.20627802690582
SVM Linear Accuracy: 98.11659192825111


In [16]:
#ensemble voting classifier
from sklearn.ensemble import VotingClassifier

names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", 
         "SGD Classifier", "Naive Bayes", "SVM Linear"]

classifiers= [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression(),
    SGDClassifier(max_iter = 100),
    MultinomialNB(),
    SVC(kernel = 'linear')
]


models = list(zip(names, classifiers))

nltk_ensemble = SklearnClassifier(VotingClassifier(estimators = models,voting ='hard', n_jobs = -1))
nltk_ensemble.train(training)
accuracy = nltk.classify. accuracy(nltk_model, testing)*100
print("Voting Classifier: Accuracy : {}".format(accuracy))



Voting Classifier: Accuracy : 98.11659192825111


In [17]:
#make class label for testing set
testing_features, labels = list(zip(*testing))

prediction = nltk_ensemble.classify_many(testing_features)


In [18]:
#print a confusion matrix and classification report
print(classification_report(labels, prediction))

pd.DataFrame(
    confusion_matrix(labels, prediction),
    index = [['actual', 'actual'], ['ham', 'spam']],
    columns = [['predicted', 'predicted'], ['ham', 'spam']])

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       970
           1       1.00      0.89      0.94       145

    accuracy                           0.99      1115
   macro avg       0.99      0.94      0.97      1115
weighted avg       0.99      0.99      0.99      1115



predicted     
                  ham spam
actual ham        970    0
       spam        16  129